# Automatic Monthly Attendance Data Extraction and Management Tool

## Introduction


> The Automatic Monthly Attendance Data Extraction and Management Tool is a Python script designed to streamline the process of extracting monthly attendance data from multiple Microsoft Word documents (docx) and organizing it into a structured format for easy management and storage. This tool is meant to be used for the WADS attendance system but can be altered to accommodate Other systems using Google Drive.


## Features:

### Google Drive Integration:

> The script integrates with Google Drive to access and save files, making it convenient for users who store their attendance data in the cloud.

### User Interaction:
> The program interacts with the user to gather the necessary input, such as the name of the individual timesheet file (docx format), which should be consistent across all participant folders.

### File Existence Check:

> The script checks the existence of the specified timesheet file in all participant folders. If the file is missing in some folders, it provides the user with a list of folders where the file is absent.

### User Confirmation:

> The user is given the option to continue with the data extraction process even if some files are missing. This feature allows for flexibility when certain participants may not have submitted their timesheets.

### Data Extraction:

> The tool extracts data from the specified timesheet file in each participant folder. It parses the tables in the document and stores the relevant information in a structured format.

### Attendance Classification:

> The program classifies attendance for each date as "Attended" or "Not Attended" based on the presence of time entries in the timesheet. It handles variations in time entry formats and accounts for missing or invalid dates.

### Data Aggregation:

> Extracted attendance data for each participant is aggregated into a single DataFrame, making it easy to manage and analyze attendance records for multiple individuals.

### Excel Export:

> The tool exports the aggregated attendance data to an Excel file. The filename is generated based on the month and year of the attendance data, ensuring organized storage.

### Google Drive Saving:

> The Excel file is saved to a specified folder in Google Drive. If a file with the same name already exists in that folder, the script provides the option to replace it or skip the saving process.

### User-Friendly Interface:

> The program includes user-friendly prompts and messages to guide the user through the process, making it accessible to individuals with varying levels of technical expertise.

## Usage:



1.   Run the cell containing '!pip install python-docx' to install the necessary dependencies.

2.   Mount Google Drive: The script begins by mounting Google Drive to access files.

3. Enter File Name: The user is prompted to enter the name of the individual timesheet file (including the .docx extension) that should be consistent across all participant folders.

4. Check for File Existence: The script checks if the specified file exists in each participant's folder and compiles lists of confirmed and unconfirmed folders.

5. User Confirmation: If some folders do not contain the specified file, the user is asked whether they want to continue with the data extraction process.

6. Data Extraction: The script extracts attendance data from the specified timesheet file in each participant folder and classifies attendance.

7. Data Aggregation: Extracted data is aggregated into a single DataFrame for all participants.

8. Excel Export: The aggregated data is exported to an Excel file with a filename based on the month and year.

9. Google Drive Saving: The Excel file is saved to a specified folder in Google Drive. The user is given the option to replace existing files.

# Conclusion:

> The Automatic Monthly Attendance Data Extraction and Management Tool simplifies the process of handling attendance data for organizations and institutions with multiple participants. It offers convenience, data organization, and user-friendly interactions, making it a valuable asset for efficient attendance management.

Note: Ensure that the necessary Python libraries (pandas, docx, Google Colab dependencies) are installed to run the script successfully.





In [ ]:
!pip install python-docx

In [ ]:


#Import Dependencies
import pandas as pd
from docx import Document
from google.colab import drive
import os
from datetime import datetime
import shutil


def save_to_drive(attendance_df):
  # Create a file name based on the date
  date_column = attendance_df.columns[0]# Assuming the date is in the first column
  # Extract the date from the date string
  date_str = date_column  # Date format is 'MM/DD/YYYY'
  date_obj = datetime.strptime(date_str, '%m/%d/%Y')
  # Create the file name using the month and year
  month_name = date_obj.strftime('%B')  # Extract month name
  year = date_obj.strftime('%Y')  # Extract year
  file_name = f"{month_name}_{year}_Attendance.xlsx"

  # Define the file path for the Excel file
  excel_file_path = os.path.join('/content', file_name)

  # Save the DataFrame to an Excel file with the new file name
  attendance_df.to_excel(excel_file_path)

  # Define the destination folder in Google Drive
  destination_folder = '/content/drive/My Drive/Monthly Attendance/'

  check_ls = os.listdir(destination_folder)#list of files in monthly attendance folder

  if file_name in check_ls:
    #if the file already exist
    # if the file already exist ask user if they wish to replace the existing file or to stop and not save the file
    #if failed to save return -1
    user_choice = input(f"The file '{file_name}' already exists in Google Drive. Do you want to replace it? (yes/no): ")
    if user_choice.lower() == 'yes':
      # Define the destination file path in Google Drive
      destination_file_path = os.path.join(destination_folder, file_name)
      # Move the Excel file to Google Drive
      shutil.move(excel_file_path, destination_file_path)
      # Inform the user that the file has been replaced in Google Drive
      print(f"Attendance data replaced in Google Drive at: {destination_file_path}")
      return 0
    else:
      # User chose not to replace the file, so we don't save it
      os.remove(excel_file_path)  # Remove the newly created Excel file
      print(f"Attendance data not saved to Google Drive.")
      return -1


  else:
    # Define the destination file path in Google Drive
    destination_file_path = os.path.join(destination_folder, file_name)

    # Move the Excel file to Google Drive
    shutil.move(excel_file_path, destination_file_path)

    # Inform the user that the file has been saved to Google Drive
    print(f"Attendance data saved to Google Drive at: {destination_file_path}")
    return 0



def individual_Data(participant_table):
  ## Function Purpose: Takes in a given table from a docx file and returns a list containing information of that table
  ## Parameters: participant_table - Table consisting of interested information. Has to be the correct format using the docx library and Document()function
  ## Return : Returns a list containing information of the table in list format

  data = []

  for row in participant_table.rows:
    row_data = []
    for cell in row.cells:
      row_data.append(cell.text)
    data.append(row_data)

  return data


def extract_helper(file_name):
  timesheets_folder_path = '/content/drive/My Drive/Individual Timesheets/'

  participant_folders = os.listdir(timesheets_folder_path)
  #dictionary containing the attendance information will be put here
  dic_of_dataframes ={}
  attendance_data = {}
  for participant_name in participant_folders:
    participant_folder_path = os.path.join(timesheets_folder_path, participant_name)

    # Check if it's a directory
    if os.path.isdir(participant_folder_path):
      # List files in the participant's folder
      participant_files = os.listdir(participant_folder_path)

      if file_name in participant_files:
        doc = Document(os.path.join(participant_folder_path,file_name))

        participant_table = doc.tables[0]#access the first table

        temp_var = individual_Data(participant_table)# contains data of individual

        dic_of_dataframes[participant_name] = temp_var

  new_df = pd.DataFrame.from_dict(dic_of_dataframes, orient='index')
  # Iterate through rows in your data
  for index, row in new_df.iterrows():#index = participant_name essentially
    attendance = {}# Dictionary to store attendance for this participant

    for cell in row:
      if isinstance(cell, list):
        date = cell[1]

        #check if the date is valid
        try:
          datetime.strptime(date,'%m/%d/%Y')
        except ValueError:
          #Invalid Date format, skip cell
          continue

        # Check if there's a valid time in the third section of the list
        time_str = cell[2] if len(cell) > 2 else ''# Get the value in the third section or an empty string if it doesn't exist

        if time_str and time_str[0].isdigit(): #time_str: It checks if time_str is not empty or blank. If it's empty, the condition evaluates to False, and the code block doesn't execute. If it's not empty, it evaluates to True.
          attendance[date] = "Attended"

        else:
          attendance[date] = "Not Attended"

    # Add the participant's attendance to the dictionary
    attendance_data[index] = attendance

  # Create a DataFrame from the attendance data
  attendance_df = pd.DataFrame.from_dict(attendance_data, orient='index')
  # Fill any remaining NaN values with "Not Attended"
  attendance_df.fillna("Not Attended", inplace=True)

  return attendance_df





def extract_checker_docx():
    confirmed_files = []  # initialize list containing all folders that have the file_name
    unconfirmed_files = []  # initialize list containing all folders that do not have the file_name
    timesheets_folder_path = '/content/drive/My Drive/Individual Timesheets/'

    file_name = input("Please Enter The Individual Time Sheet File Name Including The .docx (ex. 9-2023 Participant Timesheet.docx): ")

    while not file_name.endswith('.docx'):
        print("\nThe file name you entered has an incorrect format. Please make sure to include the correct extension (.docx).\n")
        file_name = input("Enter The File Name Again: ")

    # Read the list of subfolders (participant names)
    participant_folders = os.listdir(timesheets_folder_path)

    for participant_name in participant_folders:
        participant_folder_path = os.path.join(timesheets_folder_path, participant_name)

        if os.path.isdir(participant_folder_path):
            participant_files = os.listdir(participant_folder_path)

            if file_name in participant_files:
                confirmed_files.append(participant_name)
            else:
                unconfirmed_files.append(participant_name)

    if unconfirmed_files:
        print("\nIt seems the file does not exist in all folders or does not exist.")
        print("Here are the folders that did not have the file you entered:")
        print(unconfirmed_files)
        print("\nHere are the folders that do contain the file you entered:")
        print(confirmed_files)

        cmd = input("\nWould you like to continue regardless? Enter 'yes' to continue or 'no' to end: ").lower()

        if cmd == 'yes':
            final_df = extract_helper(file_name)
            return final_df
        else:
            return None  # Return None when the user chooses to end the program
    else:
        final_df = extract_helper(file_name)
        return final_df



if __name__ == "__main__":
    # Mount Google Drive
    drive.mount('/content/drive')

    # Call your main function to extract attendance data
    final_df = extract_checker_docx()

    if final_df is None:
        # User chose to end the program
        print("Program terminated.")
    else:
        if isinstance(final_df, pd.DataFrame):
            # User chose to continue, save the extracted attendance data to Google Drive
            save_to_drive(final_df)
        else:
            print("Invalid response from extract_checker_docx function. Program terminated.")